### Experiments with Simulated Data
Suppose we have $n=100$ training data points and another $50$ testing data points

Discrete Variables $X={X_1,X_2,X_3,X_4}$; $X_i \in \{0,1\}$; $X\in \mathcal{R}^{n\times 4}$

Continous Variable $\mathbf{z}$; $\mathbf{z} \in \mathcal{R}^{n \times 1}$

Continous Target Variable $\mathbf{y}$; $\mathbf{y} \in \mathcal{R}^{n \times 1}$

True Moderator Variable $X_1,X_2$

Data generated :
    $$y = z\beta_i+b_i+\epsilon$$

In [2]:
import numpy as np
from sklearn import linear_model
import pickle
import matplotlib.pyplot as plt
import itertools

#### Generating simulation data

In [ ]:
# Xi ~ Ber(n,pi), z ~N(0,1)
probs = [0.4,0.6,0.4,0.6]
n = 150
X = np.zeros((n,4))
for i,p in enumerate(probs):
    X[:,i]=np.array([np.random.binomial(1,p) for j in range(n)])
z = np.random.randn(n).reshape(n,1)
y = np.zeros(n)
features = np.hstack([X,z])
beta = np.random.randn(4,2)*2
print beta

In [ ]:
# y = z*beta_0+beta_1
for i in range(n):
    if X[i][0]==0:
        if X[i][1]==0: y[i] = beta[0][0]*z[i]+beta[0][1]
        else: y[i] = beta[1][0]*z[i]+beta[1][1]
    else:
        if X[i][1]==0: y[i] = beta[2][0]*z[i]+beta[2][1]
        else: y[i] = beta[3][0]*z[i]+beta[3][1]
# add noise
y = y+np.random.randn(n)*0.5

In [ ]:
# with open('data.pkl','w') as f:
#     pickle.dump((features,y),f)
# with open('coeff.pkl','w') as f:
#     pickle.dump(beta,f)

### Fit with MMR 
#### True model(including $X_1,X_2$ as moderator):
$$y = w_0 X_1+w_1 X_2+w_2Z+w_3ZX_1+w_4ZX_2+w_5ZX_1X_2+b+\epsilon$$

In [ ]:
from sklearn.linear_model import LinearRegression as lr
import matplotlib.pyplot as plt

In [ ]:
with open('data.pkl','r') as f:
    D,y = pickle.load(f)
with open('coeff.pkl','r') as f:
    true_coeff = pickle.load(f)
print true_coeff

In [ ]:
#### Reformat data
X = np.vstack([D[:,0],D[:,1],D[:,4],D[:,4]*D[:,0],D[:,4]*D[:,1],D[:,4]*D[:,0]*D[:,1]]).T
print X.shape

In [ ]:
train_X = X[:100]
test_X = X[100:]
train_y = y[:100]
test_y = y[100:]
plt.hist(train_y)
plt.show()

In [ ]:
model = lr()
model.fit(train_X,train_y)
print 'Baseline(sample mean):  {}'.format(np.mean((test_y-np.ones(50)*np.mean(train_y))**2))
print 'Training error:  {}'.format(np.mean((model.predict(train_X)-train_y)**2))
print 'Testing error:  {}'.format(np.mean((model.predict(test_X)-test_y)**2))

### Fit with Multiple Linear Regression (including $Xs = {X_1,X_2,X_3,X_4}$ as moderator)
We see MLR is overfitting since we include too many features

In [ ]:
from itertools import product

In [ ]:
inds = list(product([0,1],repeat=4))

In [ ]:
# introducint product terms
X = np.copy(D[:,:4])
for i in inds:
    Xprod = np.prod(D[:,np.nonzero(np.array(i))[0]],axis=1)
    X = np.hstack([X,(D[:,-1]*Xprod).reshape(-1,1)])

In [ ]:
train_X = X[:100]
test_X = X[100:]
train_y = y[:100]
test_y = y[100:]
plt.hist(train_y)
plt.show()

If we include all $X_i$ as indicators, we see multiple linear regression(MLR) is **overfitting** 

since testing error is almost **twice** as large as training error 

In [ ]:
model = lr()
model.fit(train_X,train_y)
print 'Baseline(sample mean):  {}'.format(np.mean((test_y-np.mean(train_y))**2))
print 'Training error:  {}'.format(np.mean((model.predict(train_X)-train_y)**2))
print 'Testing error:  {}'.format(np.mean((model.predict(test_X)-test_y)**2))

### Fit with Lasso (including $Xs = {X_1,X_2,X_3,X_4}$ as moderator)

In [ ]:
from sklearn import linear_model

In [ ]:
alphas = np.linspace(0.01,0.5,10)
train_err = []
test_err = []
coeffs = []
for a in alphas:
    model = linear_model.Lasso(alpha=a)
    model.fit(train_X,train_y)
    train_err.append(np.mean((model.predict(train_X)-train_y)**2))
    test_err.append(np.mean((model.predict(test_X)-test_y)**2))    
    coeffs.append(model.coef_)

In [ ]:
plt.plot(alphas,train_err,label = 'Training Error')
plt.plot(alphas,test_err,label='Testing Error')
plt.legend()
plt.show()

In [ ]:
terms = np.array(['X1','X2','X3','X4','Z','ZX4','ZX3','ZX3X4',\
        'ZX2','ZX2X4','ZX2X3','ZX2X3X4',\
        'ZX1','ZX1X4','ZX1X3','ZX1X3X4',\
        'ZX1X2','ZX1X2X4','ZX1X2X3','ZX1X2X3X4'])
print 'nonzero product terms:'
for coeff in coeffs:
    print terms[np.nonzero(coeff)[0]]

### Experiments(Discrete Features)

X_1,X_2 are related dependent variables
#### Submodular

In [3]:
# Data generation
# Xi ~ Ber(n,pi)

probs = [0.5,0.5,0.5,0.5,0.5,0.5]
n = 10000
X = np.zeros((n,6))
for i,p in enumerate(probs):
    X[:,i]=np.array([np.random.binomial(1,p) for j in range(n)])
# for i in range(n):
#    X[i, 0] = i & 1
#    X[i, 1] = (i & 2)>>1
#    X[i, 2] = (i & 4)>>2
#    X[i, 3] = (i & 8)>>3
#    X[i, 4] = (i & 16)>>4
#    X[i, 5] = (i & 32)>>5

# X = []
# for x0 in range(2):
#     for x1 in [0,1,1]:
#         for x2 in range(2):
#             for x3 in range(2):
#                 for x4 in range(2):
#                     for x5 in [0,1,1]:
#                         for i in range(10):
#                             X.append([x0,x1,x2,x3,x4,x5])
# X = np.array(X)
# n = len(X)


    
#print(X[-10:, :])

#asds

beta = np.random.randn(4,2)*2
y = np.zeros(n)
for i in range(n):
    if X[i][0]==0:
        if X[i][1]==0: y[i] = sum(beta[0][:2]*X[i][:2])+beta[0][1]
        else: y[i] = sum(beta[1][:2]*X[i][:2])+beta[1][1]
    else:
        if X[i][1]==0: y[i] = sum(beta[2][:2]*X[i][:2])+beta[2][1]
        else: y[i] = sum(beta[3][:2]**X[i][:2])+beta[3][1]
# add noise e~N(0,0.5^2)
y = y+np.random.randn(n)*0.5
train_X = X#[:int(n*0.8)]
test_X = X[int(n*0.8):]
train_y = y#[:int(n*0.8)]
test_y = y[int(n*0.8):]

In [5]:
# fit linear regression with given features 
def lr(features,train_X,train_y,test_X,test_y):
    features = np.array(list(features))
    n_feature =len(features)
    train_err = 0.0
    test_err = 0.0
    train_vector =np.sum(train_X[:,features]* np.array([2**i for i in range(n_feature)]),axis=1)
    test_vector =np.sum(test_X[:,features]* np.array([2**i for i in range(n_feature)]),axis=1)
    #print(train_X)
    #print(train_vector)
    for v in range(2**n_feature):
        train_idx = np.where(train_vector==v)[0]
        #print(np.where(train_vector==v))
        test_idx = np.where(test_vector==v)[0]
        train_err += np.sum((np.mean(train_y[train_idx])-train_y[train_idx])**2)
        if len(test_idx)!=0:
            test_err += np.sum((np.mean(test_y[test_idx])-test_y[test_idx])**2)
    return train_err/len(train_y),test_err/len(test_y)

In [ ]:
train_r = np.zeros(6)
test_r = np.zeros(6)
r = np.zeros(5)
feature_set = set(np.arange(6))
train_err,test_err=lr(feature_set,train_X,train_y,test_X,test_y)
print train_err,test_err
train_r[0] = train_err
test_r[0] = test_err
removed = np.zeros(5)
for i in range(5):
    dr = 100
    f = 0
    for s in feature_set:
        feature_set.remove(s)
        train_err,test_err=lr(feature_set,train_X,train_y,test_X,test_y)
        print feature_set,train_err,test_err
        if train_err-train_r[i]<dr: 
            dr = train_err-train_r[i]
            train_r[i+1] = train_err
            test_r[i+1] = test_err
            f = s;
        feature_set.add(s)
    r[i] = train_r[i+1]-train_r[i]
    feature_set.remove(f)
    removed[i]=f
print r,removed
plt.plot(np.arange(6),train_r,label='Training Error')
plt.plot(np.arange(6),test_r,label = 'Testing Error')
plt.legend()
plt.show()

#### Lasso

In [ ]:
def inter_terms(n,X):
    inds = list(itertools.product([0,1],repeat=n))
    ind_term = np.array(['X'+str(i+1) for i in range(6)])
    terms = []
    for i in inds:
        terms.append(''.join(ind_term[np.nonzero(np.array(i))[0]]))    
        Xprod = np.prod(X[:,np.nonzero(np.array(i))[0]],axis=1)
        X = np.hstack([X,Xprod.reshape(-1,1)])
    return X,terms[1:]
train_X,terms = inter_terms(6,train_X)
test_X,terms = inter_terms(6,test_X)

In [ ]:
alphas = np.linspace(0.01,0.5,10)
train_err = []
test_err = []
coeffs = []
for a in alphas:
    model = linear_model.Lasso(alpha=a)
    model.fit(train_X,train_y)
    train_err.append(np.mean((model.predict(train_X)-train_y)**2))
    test_err.append(np.mean((model.predict(test_X)-test_y)**2))    
    coeffs.append(model.coef_)

In [ ]:
plt.plot(alphas,train_err,label = 'Training Error')
plt.plot(alphas,test_err,label='Testing Error')
plt.legend()
plt.show()

In [ ]:
terms = np.array(terms)
for coeff in coeffs:
    print terms[np.nonzero(coeff)[0]]

#### Diminish Return Test for Submodularity 
$f(X \cup \{x\})-f(X)\geq f(Y \cup \{x\})-f(Y)\quad \text{if } X\subseteq Y\quad x\in \Omega \setminus Y$

In [6]:
#Y = {0, 1, 2}; X={0, 2}; x = 5
#Y = {0, 1, 2, 5}; X={0, 2, 5}
#0.000684585012625 0.0479938303236
#0.253249072143 0.252564487131
#3.23155938388 3.18356555356

# fit linear regression with given features 
def llr(features,train_X,train_y,test_X,test_y):
    features = np.array(list(features))
    features = np.array([0,2])
    n_feature =len(features)
    train_err = 0.0
    test_err = 0.0
    train_vector =np.sum(train_X[:,features]* np.array([2**i for i in range(n_feature)]),axis=1)
    test_vector =np.sum(test_X[:,features]* np.array([2**i for i in range(n_feature)]),axis=1)
    #print(train_X)
    #print(train_vector)
    var_exp = 0.0
    exp_var = 0.0
    for v in range(2**n_feature):
        train_idx = np.where(train_vector==v)[0]
        test_idx = np.where(test_vector==v)[0]
        train_err = np.sum((np.mean(train_y[train_idx])-train_y[train_idx])**2)
        
        if len(test_idx)!=0:
            test_err += np.sum((np.mean(test_y[test_idx])-test_y[test_idx])**2)
        
        train_idx5_0 = np.where((train_vector==v) & (train_X[:, 5]==0))[0]
        train_idx5_1 = np.where((train_vector==v) & (train_X[:, 5]==1))[0]
        train_idx15_00 = np.where((train_vector==v) & (train_X[:, 5]==0) & (train_X[:, 1]==0))[0]
        train_idx15_10 = np.where((train_vector==v) & (train_X[:, 5]==0) & (train_X[:, 1]==1))[0]
        train_idx15_01 = np.where((train_vector==v) & (train_X[:, 5]==1) & (train_X[:, 1]==0))[0]
        train_idx15_11 = np.where((train_vector==v) & (train_X[:, 5]==1) & (train_X[:, 1]==1))[0]    

        len5_0 = len(train_idx5_0)
        len5_1 = len(train_idx5_1)
        mean5_0 = np.mean(train_y[train_idx5_0])
        mean5_1 = np.mean(train_y[train_idx5_1])
        mean5 = (mean5_0 * len5_0 + mean5_1 * len5_1) / (len5_0 + len5_1)
        var5 = (len5_0 * (mean5_0 - mean5)**2 + len5_1 * (mean5_1 - mean5)**2) / (len5_0 + len5_1)
        
        len15_00 = len(train_idx15_00)
        len15_01 = len(train_idx15_01)
        mean15_00 = np.mean(train_y[train_idx15_00])
        mean15_01 = np.mean(train_y[train_idx15_01])
        mean1_0 = (mean15_00 * len15_00 + mean15_01 * len15_01) / (len15_00 + len15_01)
        var1_0 = (len15_00*(mean15_00-mean1_0)**2 + len15_01*(mean15_01-mean1_0)**2) / (len15_00+len15_01)
        
        len15_10 = len(train_idx15_10)
        len15_11 = len(train_idx15_11)
        mean15_10 = np.mean(train_y[train_idx15_10])
        mean15_11 = np.mean(train_y[train_idx15_11])
        mean1_1 = (mean15_10 * len15_10 + mean15_11 * len15_11) / (len15_10 + len15_11)
        var1_1 = (len15_10*(mean15_10-mean1_1)**2 + len15_11*(mean15_11-mean1_1)**2) / (len15_10+len15_11)
        
        print(var5, var1_0, var1_1)
        print((var1_0 * (len15_00+len15_01) + var1_1 * (len15_10+len15_11))/(len(train_idx)))
        #print(len15_00, len15_01, len15_10, len15_11)
        print(mean5_0, mean5_1, mean15_00, mean15_01, mean15_10, mean15_11)
        print(len5_0, len5_1, len15_00, len15_01, len15_10, len15_11)

        
              
        #asdsd

    print(var_exp/len(train_y))
    print(exp_var/len(train_y))
    print((var_exp+exp_var)/len(train_y))
    
    return train_err/len(train_y),test_err/len(test_y)

print(llr(set([0,1,2]),train_X,train_y,test_X,test_y)[0])

print(lr(set([0,1,2]),train_X,train_y,test_X,test_y)[0])
print(lr(set([0,1,2,5]),train_X,train_y,test_X,test_y)[0])
print(lr(set([0,2]),train_X,train_y,test_X,test_y)[0])
print(lr(set([0,2,5]),train_X,train_y,test_X,test_y)[0])

print(lr(set([0,1,2]),train_X,train_y,test_X,test_y)[0] - lr(set([0,1,2,5]),train_X,train_y,test_X,test_y)[0])

#asdsada


for i in range(1,4):
    s_y = list(itertools.combinations(np.arange(6),i+2))
    for s in s_y:
        s_x = list(itertools.combinations(s,i+1))
        comp = set(np.arange(6)).difference(set(s))
        for sub_s_x in s_x:
            for c in comp:
                s = set(s)
                sub_s_x = set(sub_s_x )
                print('Y = {}; X={}; x = {}'.format(s, sub_s_x,c))
                train_err1 = lr(s,train_X,train_y,test_X,test_y)[0]
                train_err2 = lr(sub_s_x,train_X,train_y,test_X,test_y)[0]
                s.add(c)
                sub_s_x.add(c)
                print('Y = {}; X={}'.format(s, sub_s_x))
                print(train_err1 - lr(s,train_X,train_y,test_X,test_y)[0],\
                  train_err2 - lr(sub_s_x,train_X,train_y,test_X,test_y)[0])
                if ((train_err1 - lr(s,train_X,train_y,test_X,test_y)[0]) 
                    - (train_err2 - lr(sub_s_x,train_X,train_y,test_X,test_y)[0])<0):
                    print('@@@@@@@@@@@@@@@@@@@')
#                     asdasdsd
                    
                #print(train_err1, lr(s,train_X,train_y,test_X,test_y)[0])
                #print(train_err2, lr(sub_s_x,train_X,train_y,test_X,test_y)[0])
                s.remove(c)
                sub_s_x.remove(c)

(1.9692302543198528e-06, 0.00014103935952521668, 2.603239105638098e-05)
8.43976123976e-05
(0.49945499552402378, 0.49664760855823464, 1.4679485444888629, 1.4441850322285954, -0.4836145433081791, -0.49382031327194792)
(1205, 1264, 607, 646, 598, 618)
(1.4668713121157401e-06, 7.8100473003289193e-05, 0.00042956709607137406)
0.00025978728453
(4.0761173227209131, 4.0736947700308406, -0.52723965956276286, -0.54491573325307618, 8.4048627424821714, 8.3634042223233269)
(1236, 1273, 599, 613, 637, 660)
(1.6215127039807483e-05, 0.00025760770061162403, 1.2657018705443421e-05)
0.000137171129037
(0.48399468107812699, 0.4920490965305726, 1.4524293313671257, 1.4203290073431147, -0.48910337844179774, -0.49622169287189549)
(1249, 1214, 626, 626, 623, 588)
(0.0073183526220201977, 5.5631945666528256e-05, 4.1910236072634978e-07)
2.7885280371e-05
(4.0477672703970704, 3.8766206557392593, -0.49669466231684362, -0.5116259699094059, 8.3788741827957782, 8.3801689701607653)
(1248, 1311, 609, 664, 639, 647)
0.0
0.0

@@@@@@@@@@@@@@@@@@@
Y = set([0, 3, 4]); X=set([0, 4]); x = 2
Y = set([0, 2, 3, 4]); X=set([0, 2, 4])
(0.01010530610079563, 0.005669534938425258)
Y = set([0, 3, 4]); X=set([0, 4]); x = 5
Y = set([0, 3, 4, 5]); X=set([0, 4, 5])
(0.0013887801316307957, 0.0013146209296355238)
Y = set([0, 3, 4]); X=set([3, 4]); x = 1
Y = set([0, 1, 3, 4]); X=set([1, 3, 4])
(10.495277986178625, 3.2770119324076621)
Y = set([0, 3, 4]); X=set([3, 4]); x = 2
Y = set([0, 2, 3, 4]); X=set([2, 3, 4])
(0.01010530610079563, 0.011351147823711827)
@@@@@@@@@@@@@@@@@@@
Y = set([0, 3, 4]); X=set([3, 4]); x = 5
Y = set([0, 3, 4, 5]); X=set([3, 4, 5])
(0.0013887801316307957, 0.0007252558669783582)
Y = set([0, 3, 5]); X=set([0, 3]); x = 1
Y = set([0, 1, 3, 5]); X=set([0, 1, 3])
(10.497253988808408, 10.498143727388719)
@@@@@@@@@@@@@@@@@@@
Y = set([0, 3, 5]); X=set([0, 3]); x = 2
Y = set([0, 2, 3, 5]); X=set([0, 2, 3])
(0.0085432437892318802, 0.0044475452772925195)
Y = set([0, 3, 5]); X=set([0, 3]); x = 4
Y = set([0, 3, 4, 5])

Y = set([1, 4, 5]); X=set([1, 5]); x = 0
Y = set([0, 1, 4, 5]); X=set([0, 1, 5])
(10.324241664003178, 10.331471917013783)
@@@@@@@@@@@@@@@@@@@
Y = set([1, 4, 5]); X=set([1, 5]); x = 2
Y = set([1, 2, 4, 5]); X=set([1, 2, 5])
(0.009166236628111335, 0.0011339274948927169)
Y = set([1, 4, 5]); X=set([1, 5]); x = 3
Y = set([1, 3, 4, 5]); X=set([1, 3, 5])
(0.007234382564393016, 0.0005381309598977424)
Y = set([1, 4, 5]); X=set([4, 5]); x = 0
Y = set([0, 1, 4, 5]); X=set([0, 4, 5])
(10.324241664003178, 3.105179084018868)
Y = set([1, 4, 5]); X=set([4, 5]); x = 2
Y = set([1, 2, 4, 5]); X=set([2, 4, 5])
(0.009166236628111335, 0.0067658610722993728)
Y = set([1, 4, 5]); X=set([4, 5]); x = 3
Y = set([1, 3, 4, 5]); X=set([3, 4, 5])
(0.007234382564393016, 0.0020332375873071129)
Y = set([2, 3, 4]); X=set([2, 3]); x = 0
Y = set([0, 2, 3, 4]); X=set([0, 2, 3])
(3.101947449803566, 3.1063234158778084)
@@@@@@@@@@@@@@@@@@@
Y = set([2, 3, 4]); X=set([2, 3]); x = 1
Y = set([1, 2, 3, 4]); X=set([1, 2, 3])
(3.2798

Y = set([0, 1, 3, 4, 5]); X=set([0, 1, 3, 4])
(0.00044646375057227905, 0.00018953307361330163)
Y = set([0, 1, 3, 5]); X=set([0, 1, 5]); x = 2
Y = set([0, 1, 2, 3, 5]); X=set([0, 1, 2, 5])
(0.00029565032730152763, 0.00012083423344638744)
Y = set([0, 1, 3, 5]); X=set([0, 1, 5]); x = 4
Y = set([0, 1, 3, 4, 5]); X=set([0, 1, 4, 5])
(0.00044646375057227905, 8.6566024142809006e-05)
Y = set([0, 1, 3, 5]); X=set([0, 3, 5]); x = 2
Y = set([0, 1, 2, 3, 5]); X=set([0, 2, 3, 5])
(0.00029565032730152763, 0.0085432437892318802)
@@@@@@@@@@@@@@@@@@@
Y = set([0, 1, 3, 5]); X=set([0, 3, 5]); x = 4
Y = set([0, 1, 3, 4, 5]); X=set([0, 3, 4, 5])
(0.00044646375057227905, 0.0034268407622857211)
@@@@@@@@@@@@@@@@@@@
Y = set([0, 1, 3, 5]); X=set([1, 3, 5]); x = 2
Y = set([0, 1, 2, 3, 5]); X=set([1, 2, 3, 5])
(0.00029565032730152763, 0.0023721026704954795)
@@@@@@@@@@@@@@@@@@@
Y = set([0, 1, 3, 5]); X=set([1, 3, 5]); x = 4
Y = set([0, 1, 3, 4, 5]); X=set([1, 3, 4, 5])
(0.00044646375057227905, 0.014013070639242287

Y = set([1, 2, 4, 5]); X=set([1, 2, 4]); x = 0
Y = set([0, 1, 2, 4, 5]); X=set([0, 1, 2, 4])
(10.315378790495188, 10.319129737971014)
@@@@@@@@@@@@@@@@@@@
Y = set([1, 2, 4, 5]); X=set([1, 2, 4]); x = 3
Y = set([1, 2, 3, 4, 5]); X=set([1, 2, 3, 4])
(0.014453529351614591, 0.011883681628798826)
Y = set([1, 2, 4, 5]); X=set([1, 2, 5]); x = 0
Y = set([0, 1, 2, 4, 5]); X=set([0, 1, 2, 5])
(10.315378790495188, 10.330458823752336)
@@@@@@@@@@@@@@@@@@@
Y = set([1, 2, 4, 5]); X=set([1, 2, 5]); x = 3
Y = set([1, 2, 3, 4, 5]); X=set([1, 2, 3, 5])
(0.014453529351614591, 0.001776306135500505)
Y = set([1, 2, 4, 5]); X=set([1, 4, 5]); x = 0
Y = set([0, 1, 2, 4, 5]); X=set([0, 1, 4, 5])
(10.315378790495188, 10.324241664003178)
@@@@@@@@@@@@@@@@@@@
Y = set([1, 2, 4, 5]); X=set([1, 4, 5]); x = 3
Y = set([1, 2, 3, 4, 5]); X=set([1, 3, 4, 5])
(0.014453529351614591, 0.007234382564393016)
Y = set([1, 2, 4, 5]); X=set([2, 4, 5]); x = 0
Y = set([0, 1, 2, 4, 5]); X=set([0, 2, 4, 5])
(10.315378790495188, 3.10598767

In [ ]:
#Y = {0, 1, 2}; X={0, 2}; x = 5
#Y = {0, 1, 2, 5}; X={0, 2, 5}

# fit linear regression with given features 
def llr(features,train_X,train_y,test_X,test_y):
    features = np.array(list(features))
    features = np.array([0,1,2])
    n_feature =len(features)
    train_err = 0.0
    test_err = 0.0
    train_vector =np.sum(train_X[:,features]* np.array([2**i for i in range(n_feature)]),axis=1)
    test_vector =np.sum(test_X[:,features]* np.array([2**i for i in range(n_feature)]),axis=1)
    #print(train_X)
    #print(train_vector)
    var_exp = 0.0
    exp_var = 0.0
    for v in range(2**n_feature):
        train_idx = np.where(train_vector==v)[0]
        test_idx = np.where(test_vector==v)[0]
        train_err += np.sum((np.mean(train_y[train_idx])-train_y[train_idx])**2)
        if len(test_idx)!=0:
            test_err += np.sum((np.mean(test_y[test_idx])-test_y[test_idx])**2)
        
        train_idx0 = np.where((train_vector==v) & (train_X[:, 5]==0))[0]
        train_idx1 = np.where((train_vector==v) & (train_X[:, 5]==1))[0]
        len0 = len(train_idx0)
        len1 = len(train_idx1)
        count = len(train_idx0) + len(train_idx1)
        mean0 = np.mean(train_y[train_idx0])
        mean1 = np.mean(train_y[train_idx1])
        mean01 = (len0 * mean0 + len1 * mean1) / (len0 + len1)
        tmp1 = (len0 * (mean0 - mean01)**2 + len1 * (mean1 - mean01)**2) / (len0 + len1)
        tmp2 = (np.var(train_y[train_idx0]) * len(train_idx0) + 
                np.var(train_y[train_idx1]) * len(train_idx1)) / count
        tmp3 = np.var(train_y[train_idx])
        
        print('!!!')
        print(tmp1, tmp2, tmp3, tmp1 + tmp2)
        #print(np.sum((np.mean(train_y[train_idx])-train_y[train_idx])**2) - tmp3 * count, '!')
        
        #var_exp += np.var([np.mean(train_y[train_idx0]), np.mean(train_y[train_idx1])]) * count
        var_exp += tmp1 * count
        exp_var += np.var(train_y[train_idx0]) * len(train_idx0) + np.var(train_y[train_idx1]) * len(train_idx1)

    print(var_exp/len(train_y))
    print(exp_var/len(train_y))
    print((var_exp+exp_var)/len(train_y))
    
    return train_err/len(train_y),test_err/len(test_y)

print(llr(set([0,1,2]),train_X,train_y,test_X,test_y)[0])

print(lr(set([0,1,2]),train_X,train_y,test_X,test_y)[0])
print(lr(set([0,1,2,5]),train_X,train_y,test_X,test_y)[0])
print(lr(set([0,2]),train_X,train_y,test_X,test_y)[0])
print(lr(set([0,2,5]),train_X,train_y,test_X,test_y)[0])

print(lr(set([0,1,2]),train_X,train_y,test_X,test_y)[0] - lr(set([0,1,2,5]),train_X,train_y,test_X,test_y)[0])

asd


for i in range(1,4):
    s_y = list(itertools.combinations(np.arange(6),i+2))
    for s in s_y:
        s_x = list(itertools.combinations(s,i+1))
        comp = set(np.arange(6)).difference(set(s))
        for sub_s_x in s_x:
            for c in comp:
                s = set(s)
                sub_s_x = set(sub_s_x )
                print('Y = {}; X={}; x = {}'.format(s, sub_s_x,c))
                train_err1 = lr(s,train_X,train_y,test_X,test_y)[0]
                train_err2 = lr(sub_s_x,train_X,train_y,test_X,test_y)[0]
                s.add(c)
                sub_s_x.add(c)
                print('Y = {}; X={}'.format(s, sub_s_x))
                print(train_err1 - lr(s,train_X,train_y,test_X,test_y)[0],\
                  train_err2 - lr(sub_s_x,train_X,train_y,test_X,test_y)[0])
                print(train_err1, lr(s,train_X,train_y,test_X,test_y)[0])
                print(train_err2, lr(sub_s_x,train_X,train_y,test_X,test_y)[0])
                s.remove(c)
                sub_s_x.remove(c)

In [ ]:
data = np.array([0.1, 0.9, 0.3, 0.2, 0.4])
tmp3 = np.var(data)
tmp1 = np.var([np.mean(data[0:3]), np.mean(data[0:3]), np.mean(data[0:3]), np.mean(data[3:5]), np.mean(data[3:5])])
tmp1 = np.var([np.mean(data[0:3])*3, np.mean(data[3:5])*2])/25
tmp2 = (np.var(data[0:3]) * 3 + np.var(data[3:5]) * 2) / 5
print(tmp1, tmp2, tmp3, tmp1 + tmp2)